In [2]:
import numpy as np
import cv2

import sys 
sys.path.append('../utils')

In [3]:
from image_processing_helper import *
from radiomics_helper import *

c:\Users\rautc\miniforge3\envs\aln_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
c:\HII Lab\Breast Cancer\Breast_Cancer_Ultrasound_Project_AIIMS\models\../utils\image_processing_helper.py:4: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [4]:
cropped_images_folder = "../data/nodes_data/december_nodes_images/Cropped_Images_Folder"
cropped_clean_images_folder = "../data/nodes_data/december_nodes_images/Cropped_Clean_Images_Folder"

In [10]:
img_path = cropped_images_folder + "/P_101_3.PNG"
img_out_path = cropped_clean_images_folder + "/P_101_2.PNG"

In [11]:
img = read_as_grayscale(img_path)
cross_mask = manual_mask_segmentation(img)

Draw the contour using left mouse button. Press 's' to finish and return the mask.


In [12]:
Image.fromarray(cross_mask).save("crossmark_mask.png")

In [13]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import numpy as np

# Load pre-trained inpainting model
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", 
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

def inpaint_ultrasound(image_path, mask_path, prompt="clean medical ultrasound image without artifacts", strength=0.99, guidance_scale=7.5):
    """
    Inpaint ultrasound image to remove crossmarks.
    
    Args:
    - image_path: Path to original ultrasound PNG/JPG.
    - mask_path: Path to mask (white=area to inpaint, black=keep) same size as image.
    - prompt: Describes desired fill (e.g., "homogeneous breast tissue ultrasound").
    - strength: Denoising strength (0.8-1.0 for strong artifacts).
    - guidance_scale: Prompt adherence (5-10).
    
    Returns: Inpainted PIL Image.
    """
    init_image = Image.open(image_path).convert("RGB").resize((512, 512))
    mask_image = Image.open(mask_path).convert("L").resize((512, 512))
    
    result = pipe(
        prompt=prompt,
        image=init_image,
        mask_image=mask_image,
        num_inference_steps=50,
        strength=strength,
        guidance_scale=guidance_scale
    ).images[0]
    
    return result

# Example usage (create mask manually or via OpenCV for crossmarks)
temp = inpaint_ultrasound(img_path, "crossmark_mask.png")


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\rautc\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\vae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\rautc\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\rautc\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\rautc\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/49 [00:00<?, ?it/s]

In [24]:
import pandas as pd
dec_lab = pd.read_csv("../data/nodes_data/december_nodes_images/december_nodes_labels.csv")
dec_lab_update = pd.read_csv("../data/nodes_data/december_nodes_images/december_nodes_labels_updated.csv")



In [25]:
dec_lab

,Unnamed: 0,S.No.,patient ID,Age,No. of nodes,Histopath,ER,PR,Her 2 neu,Ki67,...,FNAC/BIOPSY,Additional comments,Original Image Name,Image ID,Image type,Comments,original_image_path,found_path,path_exists,moved_path
0,0,1,101,27.0,1.0,IBC NST,8,6,Negative 0,25,...,fnac,NaN,Clipboard_03-15-2025_17,P_101_1,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
1,1,1,101,27.0,1.0,IBC NST,8,6,Negative 1,25,...,fnac,NaN,Clipboard_03-15-2025_18,P_101_2,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
2,2,1,101,27.0,1.0,IBC NST,8,6,Negative 2,25,...,fnac,NaN,Clipboard_03-15-2025_19,P_101_3,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
3,3,1,101,27.0,1.0,IBC NST,8,6,Negative 3,25,...,fnac,NaN,Clipboard_03-15-2025_20,P_101_4,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
4,4,1,101,27.0,1.0,IBC NST,8,6,Negative 4,25,...,fnac,NaN,Clipboard_03-15-2025_21,P_101_5,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,592,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,FNAC,NaN,IMG-0022-00008,P_195_8,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
290,593,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,FNAC,NaN,IMG-0022-00009,P_195_9,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
291,594,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,FNAC,NaN,IMG-0022-00010,P_195_10,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...
292,595,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,FNAC,NaN,IMG-0022-00011,P_195_11,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,NaN,False,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...


In [28]:
dec_lab_update = dec_lab_update[dec_lab_update['Image type'] == 'Node']

In [33]:
dec_labels = dec_lab_update.merge(dec_lab, on='Image ID', how='left', suffixes=('', '_drop'))

In [39]:
dec_labels = dec_labels.drop([col for col in dec_labels if col.endswith('_drop')], axis=1)

In [40]:
dec_labels.to_csv("../data/nodes_data/december_nodes_images/december_nodes_labels_final.csv", index=False)

In [41]:
dec_labels.columns

Index(['S.No.', 'patient ID', 'Age', 'No. of nodes', 'Histopath', 'ER', 'PR',
       'Her 2 neu', 'Ki67', 'molecularsubtype', 'histopathology',
       'FNAC/BIOPSY', 'Additional comments', 'Original Image Name', 'Image ID',
       'Image type', 'Comments', 'Copy of', 'Has Crossmarks',
       'Actual no. of nodes', 'Doppler ', 'Include or not',
       'Reason to not include', 'Unnamed: 0', 'original_image_path',
       'found_path', 'path_exists', 'moved_path'],
      dtype='object')

In [43]:
dec_labels.iloc[0]['moved_path']

'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/december_nodes_images\\P_101_1.tif'

In [44]:
import os
from glob import glob
from pathlib import Path
import pandas as pd  # only if you need it here

base_dir = Path("../data/nodes_data/december_nodes_images/Cropped_Images_Folder")

# Filter rows: Has Crossmarks == "Yes" and Include or not != "No"
mask = (dec_labels["Has Crossmarks"] == "Yes") & (dec_labels["Include or not"] != "No")
filtered = dec_labels.loc[mask]

for _, row in filtered.iterrows():
    img_id = str(row["Image ID"])

    # Match any extension: try all files whose stem matches Image ID
    pattern = str(base_dir / f"{img_id}.*")
    matches = glob(pattern)

    if not matches:
        # Optionally log or continue
        print(f"No image found for ID {img_id}")
        continue

    img_path = matches[0]  # take the first match if multiple
    cross_mask = create_rect_mask(img_path)
    cross_mask.save("../data/nodes_data/december_nodes_images/Cropped_Images_Cross_Mask_Folder" + f"/{img_id}_mask.png")


Instructions:
  Left drag: draw rectangle over crossmark
  u: undo last rectangle
  c: clear all
  s: save mask and exit
  q or ESC: quit without saving
Mask saved to crossmark_mask_rect.png
Instructions:
  Left drag: draw rectangle over crossmark
  u: undo last rectangle
  c: clear all
  s: save mask and exit
  q or ESC: quit without saving
Mask saved to crossmark_mask_rect.png
Instructions:
  Left drag: draw rectangle over crossmark
  u: undo last rectangle
  c: clear all
  s: save mask and exit
  q or ESC: quit without saving
Mask saved to crossmark_mask_rect.png
Instructions:
  Left drag: draw rectangle over crossmark
  u: undo last rectangle
  c: clear all
  s: save mask and exit
  q or ESC: quit without saving
Mask saved to crossmark_mask_rect.png
Instructions:
  Left drag: draw rectangle over crossmark
  u: undo last rectangle
  c: clear all
  s: save mask and exit
  q or ESC: quit without saving
Mask saved to crossmark_mask_rect.png
Instructions:
  Left drag: draw rectangle ove

In [45]:
for _, row in filtered.iterrows():
    img_id = str(row["Image ID"])

    # Match any extension: try all files whose stem matches Image ID
    pattern = str(base_dir / f"{img_id}.*")
    matches = glob(pattern)

    if not matches:
        # Optionally log or continue
        print(f"No image found for ID {img_id}")
        continue

    img_path = matches[0]  # take the first match if multiple
    cross_mask_path = "../data/nodes_data/december_nodes_images/Cropped_Images_Cross_Mask_Folder" + f"/{img_id}_mask.png"
    inpainted_image = inpaint_ultrasound(img_path, cross_mask_path)
    inpainted_image.save("../data/nodes_data/december_nodes_images/Cleaned_Inpainted_Images_Folder" + f"/{img_id}.png")   

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
